In [ ]:
%matplotlib ipympl

from experimental.beacon_sim.experiment_results_pb2 import ExperimentResult
from experimental.beacon_sim.world_map_config_pb2 import WorldMapConfig
from experimental.beacon_sim.mapped_landmarks_pb2 import MappedLandmarks
import planning.probabilistic_road_map_python as prmp
import itertools

import pandas as pd
import pandas.io.formats.style

from collections import defaultdict
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets
import matplotlib as mpl
import re

mpl.style.use('ggplot')

import seaborn as sns

from pathlib import Path

import numpy as np

import experimental.beacon_sim.ekf_slam_python as esp
import experimental.beacon_sim.belief_road_map_planner_python as brmp
import experimental.beacon_sim.beacon_potential_python as bpp
import planning.probabilistic_road_map_python as prmp
import common.time.robot_time_python as rtp
import experimental.beacon_sim.robot_belief_python as rbp
import common.liegroups.se2_python as se2


In [ ]:
METRIC_NAME = 'prob_mass_in_region_regret'

In [ ]:
# file_name = '/tmp/compare_against_baseline_complex_env.pb'
# file_name = '/tmp/test_timing.pb'
# results_dir = Path('/home/erick/scratch/beacon_sim/20240604/results')
# configs_dir = Path('/home/erick/scratch/beacon_sim/20240604/experiment_configs/')
results_dir = Path('/home/erick/scratch/beacon_sim/20240716/jobs/reprocess_results/results/')
configs_dir = Path('/home/erick/scratch/beacon_sim/20240716/experiment_configs')



fig_path = results_dir.parent / 'figs'
fig_path.mkdir(exist_ok=True)

results_files = sorted(results_dir.glob("*.pb"))
protos_by_exp = {}
configs_by_exp = {}
to_exclude = []
# to_exclude = [
#         'indep_p_beacon_1.00_lm_density_1.0E-02_timeout_3600',
#        'indep_p_beacon_1.00_lm_density_1.0E-03_timeout_3600',
#        'mutex_num_class_2_lm_density_1.00E-03_timeout_3600',
#        'mutex_num_class_4_lm_density_1.00E-03_timeout_3600',
#        'mutex_num_class_8_lm_density_1.00E-03_timeout_3600',
#         'indep_p_beacon_0.75_lm_density_1.0E-03_timeout_3600'
# ]

for f in results_files:
    if f.stem in to_exclude:
        continue
    with open(f, 'rb') as file_in:
        data = ExperimentResult()
        data.ParseFromString(file_in.read())
        protos_by_exp[f.stem] = data

        map_config_path = configs_dir / f.stem / data.experiment_config.map_config_path
        road_map_path = configs_dir / f.stem / data.experiment_config.road_map_path
        mapped_landmark_path = configs_dir / f.stem / data.experiment_config.ekf_state_path

        wmc = WorldMapConfig()
        wmc.ParseFromString(map_config_path.read_bytes())

        assert road_map_path.exists()
        map_proto = MappedLandmarks()
        map_proto.ParseFromString(mapped_landmark_path.read_bytes())
        configs_by_exp[f.stem] = {
            'road_map': prmp.RoadMap.from_proto_string(road_map_path.read_bytes()),
            'world_map_config': wmc,
            'mapped_landmarks': map_proto,
        }

        

In [ ]:
def protobuf_to_df(pb):
    out = defaultdict(list)
    i = 0

    for result in pb.results:
        planner_name = pb.planner_names[result.planner_id]
        m = re.match(r'(.*?) BRM\s?(\d*)', planner_name)
        out['elapsed_time'].append(result.elapsed_time.count / 1e9)
        out['trial_id'].append(result.trial_id)
        out['planner_id'].append(result.planner_id)
        out['planner_name'].append(planner_name)
        out['planner_type'].append(m.group(1))
        out['planner_size'].append(m.group(2) if len(m.group(2)) > 0 else '1')
        out['start'].append(np.array(pb.start_goal[result.trial_id].start.data))
        out['goal'].append(np.array(pb.start_goal[result.trial_id].goal.data))
        out['timeout'].append(pb.experiment_config.plan_timeout_s)
        if result.HasField('plan'):
            out['plan'].append(list(result.plan.nodes))
            out['log_prob_mass'].append(result.plan.log_prob_mass)
            out['expected_size'].append(result.plan.expected_size)
            out['expected_determinant'].append(result.plan.average_plan_metrics.expected_determinant)
            out['expected_position_determinant'].append(result.plan.average_plan_metrics.expected_position_determinant)
            out['prob_mass_in_region'].append(1-result.plan.average_plan_metrics.prob_mass_in_region)
            out['prob_mass_in_region_regret'].append(-result.plan.average_plan_metrics.prob_mass_in_region_regret)
        else:
            out['plan'].append(None)
            out['log_prob_mass'].append(None)
            out['expected_size'].append(None)
            out['expected_determinant'].append(None)
            out['expected_position_determinant'].append(None)
            out['prob_mass_in_region'].append(None)
            out['prob_mass_in_region_regret'].append(None)
    return pd.DataFrame(out)


def extract_clusters_from_potential(potential_proto):
    
    potential_case = potential_proto.WhichOneof('potential_oneof')
    if potential_case in ['correlated_beacon_potential', 'anticorrelated_potential', 'precision_matrix_potential']:
        return [list(getattr(potential_proto, potential_case).members)]  
    elif potential_case == 'combined_potential':
        out = []
        for pot in potential_proto.combined_potential.potentials:
            out.extend(extract_clusters_from_potential(pot))
        return out
    print('unknown potential type', potential_case)
    raise NotImplementedError
    ...

def plot_mapped_landmarks(mapped_landmarks, potential_proto):
    # extract clusters from potential
    clusters = extract_clusters_from_potential(potential_proto)
    
    ids = list(mapped_landmarks.beacon_ids)
    for cluster in clusters:
        xs = []
        ys = []
        for beacon_id in cluster:
            idx = ids.index(beacon_id)
            beacon_in_map = mapped_landmarks.beacon_in_local[idx]
            xs.append(beacon_in_map.data[0])
            ys.append(beacon_in_map.data[1])
            # plt.text(beacon_in_map.data[0], beacon_in_map.data[1], f't_{ids[i]}')
        plt.plot(xs, ys, 'o')

def plot_road_map(road_map, label_nodes=True):
    pts = np.stack(road_map.points())

    
    
    if road_map.has_start_goal():
        for idx, marker in [(road_map.START_IDX, 'rs'), (road_map.GOAL_IDX, 'y*')]:
            pt = road_map.point(idx)
            plt.plot(*pt, marker)
            

    line_segments = []
    for i in range(len(pts)):
        for j in range(i+1, len(pts)):
            if road_map.adj()[i, j] != 0:
                line_segments.append([pts[i, :], pts[j,:]])

    if road_map.has_start_goal():
        for idx in [road_map.START_IDX, road_map.GOAL_IDX]:
            pt = road_map.point(idx)
            neighbors = road_map.neighbors(idx)
            for _, neighbor_pt in neighbors:
                line_segments.append([pt, neighbor_pt])
    

    edges = mpl.collections.LineCollection(line_segments, colors=(0.6, 0.8, 0.6, 1.0))
    ax = plt.gca()
    ax.add_collection(edges)

    plt.plot(pts[:, 0], pts[:, 1], '*', color='tab:gray')
    if label_nodes:
        for i, pt in enumerate(pts):
                plt.text(*pt, str(i))


def plot_plan(road_map, execution_df):
    pts = [execution_df['start'].iloc[0]]
    pts += [road_map.point(x) for x in execution_df.plan.iloc[0] if x >=0]
    pts.append(execution_df['goal'].iloc[0])
    pts = np.vstack(pts)

    plt.plot(pts[:, 0], pts[:, 1], 'm')
    plt.plot(pts[0, 0], pts[0, 1], 'gs')
    plt.plot(pts[-1, 0], pts[-1, 1], 'rv')

In [ ]:
sub_dfs = []
for name, proto in protos_by_exp.items():
    sub_df = protobuf_to_df(proto)
    sub_df['exp_name'] = name
    sub_df['exp_type'] = name.split('_')[0]

    sub_dfs.append(sub_df)
df = pd.concat(sub_dfs)
df.reset_index(inplace=True)

In [ ]:
df

In [ ]:
pos_regret_mask = df['prob_mass_in_region_regret'] > 1e-6
# exp_mask  = df['exp_name'] == 'spatial_clus_dens_1.0E-03_p_beacon_0.25_p_no_beacon_0.10_lm_density_1.00E-01_timeout_3600'
pos_regret_df = df[pos_regret_mask]
print('Num trials with positive regret:', len(pos_regret_df))
max_positive_regret_idx = pos_regret_df['prob_mass_in_region_regret'].argmax()

In [ ]:
# df = df[~pos_regret_mask]

In [ ]:
print(pos_regret_df.iloc[max_positive_regret_idx])
print(pos_regret_df.iloc[max_positive_regret_idx].exp_name)
np.unique(pos_regret_df.exp_name)

In [ ]:
def make_plots(df, cdf_fig, pdf_fig, exec_time_fig):
    bins = np.linspace(0, 3, 1000)
    plt.figure(cdf_fig)
    plt.subplot(121)
    plt.cla()
    for i, g in df.groupby('planner_id'):
        plt.hist(g[METRIC_NAME], density=True, histtype='step', label=g['planner_name'].iloc[0], bins=bins, cumulative=True)
        
    plt.xlabel('Expected Determinant')
    plt.ylabel('Fraction')
    plt.legend()
    plt.suptitle(df["exp_name"].iloc[0])
    plt.tight_layout()
    

    plt.figure(pdf_fig)
    ax = plt.gca()
    plt.cla()
    axes = df.plot.hist(by='planner_name', column=METRIC_NAME, bins=50, ax=ax)
    axes[-1].set_xlabel('Expected Determinant')
    plt.suptitle('Expected Determinant')
    plt.tight_layout()
    
    plt.figure(exec_time_fig)
    ax = plt.gca()
    plt.cla()
    axes = df.plot.hist(by='planner_name', column='elapsed_time', bins=50, ax=ax)
    for i, ax in enumerate(axes):
        ax.set_yscale('log')

    axes[-1].set_xlabel('Elapsed Time (s)')
    plt.suptitle('Elasped Time')
    plt.tight_layout()

def make_trial_plot(trial_fig, trial_df):
    plt.figure(trial_fig)
    plt.subplot(122)
    plt.cla()
    exp_name = trial_df['exp_name'].iloc[0]
    exp_configs = configs_by_exp[exp_name]
    plot_mapped_landmarks(exp_configs['mapped_landmarks'], exp_configs['world_map_config'].correlated_beacons.potential)
    plot_road_map(exp_configs['road_map'])
    plot_plan(exp_configs['road_map'], trial_df)


def plot_overview_results(df):
    exps = np.unique(df['exp_name'])
    exp_selector = ipywidgets.Dropdown(options=exps, description='experiment:')
    trial_selector = ipywidgets.IntSlider(min=0, max=1, step=1, value=0, description='Trial Selector:', behavior='snap')
    planner_selector = ipywidgets.Dropdown(options=[], description='planner:')
    hbox = ipywidgets.HBox()
    cdf_fig = plt.figure(figsize=(16, 8))
    pdf_fig = plt.figure(figsize=(13,10))
    exec_time_fig = plt.figure(figsize=(13,10))
    # trial_fig = plt.figure(figsize=(8, 8))

    def on_trial_or_planner_change(change):
        exp_mask = df['exp_name'] == exp_selector.value
        trial_mask = df['trial_id'] == trial_selector.value
        planner_mask = df['planner_name'] == planner_selector.value
        mask = np.logical_and(exp_mask, trial_mask)
        execution_mask = np.logical_and(mask, planner_mask)

        trial_df = df[mask]
        # _ = display(trial_df, display_id='trial_df')
        execution_df = df[execution_mask]
        make_trial_plot(cdf_fig, execution_df)

        
    def on_instance_change(change):
        selected_exp_name = exp_selector.value
         
        sub_df = df[df['exp_name'] == selected_exp_name]

        trial_selector.min = 0
        trial_selector.max = sub_df['trial_id'].max()
        trial_selector.value = 0

        planner_selector.options = np.unique(sub_df['planner_name'])
        planner_selector.value = planner_selector.options[0]
        
        make_plots(sub_df, cdf_fig, pdf_fig, exec_time_fig)
        on_trial_or_planner_change({})

        
    
    hbox.children=(exp_selector, trial_selector, planner_selector)

    exp_selector.observe(on_instance_change, 'value')
    trial_selector.observe(on_trial_or_planner_change, 'value')
    planner_selector.observe(on_trial_or_planner_change, 'value')
    display(hbox)
    on_instance_change({})
    
   

In [ ]:
plot_overview_results(df)


# axes = df.plot.hist(by='planner_id', column='log_prob_mass', bins=50, figsize=(13,10))
# for i, ax in enumerate(axes):
#     ax.set_yscale('log')
#     ax.set_title(planners[i])
# axes[-1].set_xlabel('Log Prob Mass')
# plt.suptitle('Log Prob Mass')
# plt.tight_layout()

In [ ]:
def make_summary_plots(exps_and_locs, num_rows, num_cols, sup_title, timeout=None, to_remove=None, fig_path=None):
    if to_remove is None:
        to_remove = []

    if timeout:
        to_remove.append(f'_timeout_{timeout:02d}')
        sup_title = f"{sup_title}_timeout_{timeout:04d}"
        
    
    for exp_name, plot_id in exps_and_locs:
        if timeout:
            exp_name = f"{exp_name}_timeout_{timeout:02d}"
        plt.subplot(num_rows, num_cols, plot_id)
        exp_mask = df['exp_name'] == exp_name
        exp_df = df[exp_mask]
        if not exp_df.empty:
            ax = sns.boxplot(x='planner_type',
                        y=METRIC_NAME,
                        hue='planner_size',
                        data=exp_df,
                       legend=(plot_id==1))
                
            if plot_id == 1:
                sns.move_legend(ax, 'upper left', bbox_to_anchor=(0.1, 1.0))
        
        plot_title = exp_name
        for r in to_remove:
            plot_title = plot_title.replace(r, '')
            
        plt.title(plot_title)
        plt.xlabel('')
        plt.ylim(-1.1, 0.1)

        
    plt.suptitle(sup_title)
    plt.tight_layout()
    if fig_path:
        plt.savefig(fig_path / f"summary_{sup_title}.svg")

    ...



timeouts = [3600]

for timeout in timeouts:
    indep_exps_and_locs = [
        ('indep_p_beacon_0.10_lm_density_1.0E-03', 1),
        ('indep_p_beacon_0.10_lm_density_1.0E-02', 2),
        ('indep_p_beacon_0.25_lm_density_1.0E-03', 3),
        ('indep_p_beacon_0.25_lm_density_1.0E-02', 4),
        ('indep_p_beacon_0.50_lm_density_1.0E-03', 5),
        ('indep_p_beacon_0.50_lm_density_1.0E-02', 6),
        ('indep_p_beacon_0.75_lm_density_1.0E-03', 7),
        ('indep_p_beacon_0.75_lm_density_1.0E-02', 8),
        # ('indep_p_beacon_1.00_lm_density_1.0E-03', 9),
        ('indep_p_beacon_1.00_lm_density_1.0E-02', 10),
        
    ]
    
    plt.figure(figsize=(10,12))
    make_summary_plots(indep_exps_and_locs, num_rows=5, num_cols=2, sup_title='indep', timeout=timeout, fig_path=fig_path, to_remove=['indep_'])
        
    
    mutex_exps_and_locs = [
        ('mutex_num_class_2_lm_density_1.00E-03', 1),
        ('mutex_num_class_2_lm_density_1.00E-02', 2),
        ('mutex_num_class_4_lm_density_1.00E-03', 3),
        ('mutex_num_class_4_lm_density_1.00E-02', 4),
        ('mutex_num_class_8_lm_density_1.00E-03', 5),
        ('mutex_num_class_8_lm_density_1.00E-02', 6),
    ]
    
    plt.figure(figsize=(10,9))
    
    make_summary_plots(mutex_exps_and_locs, num_rows=3, num_cols=2, sup_title='mutex', timeout=timeout, fig_path=fig_path, to_remove=["mutex_"])
    
    high_spatial_cluster_exps_and_locs = [
    ('spatial_clus_dens_1.0E-03_p_beacon_0.10_p_no_beacon_0.10_lm_density_1.00E-01', 1),
    ('spatial_clus_dens_1.0E-03_p_beacon_0.10_p_no_beacon_0.50_lm_density_1.00E-01', 2),
    ('spatial_clus_dens_1.0E-03_p_beacon_0.25_p_no_beacon_0.10_lm_density_1.00E-01', 3),
    ('spatial_clus_dens_1.0E-03_p_beacon_0.25_p_no_beacon_0.50_lm_density_1.00E-01', 4),
    ('spatial_clus_dens_1.0E-03_p_beacon_0.50_p_no_beacon_0.10_lm_density_1.00E-01', 5),
    ('spatial_clus_dens_1.0E-03_p_beacon_0.50_p_no_beacon_0.50_lm_density_1.00E-01', 6),
    ('spatial_clus_dens_1.0E-03_p_beacon_0.75_p_no_beacon_0.10_lm_density_1.00E-01', 7),
    ('spatial_clus_dens_1.0E-03_p_beacon_0.75_p_no_beacon_0.50_lm_density_1.00E-01', 8),
    ('spatial_clus_dens_1.0E-03_p_beacon_1.00_p_no_beacon_0.10_lm_density_1.00E-01', 9),
    ('spatial_clus_dens_1.0E-03_p_beacon_1.00_p_no_beacon_0.50_lm_density_1.00E-01', 10),
    ]
    
    plt.figure(figsize=(10,12))
    make_summary_plots(high_spatial_cluster_exps_and_locs, num_rows=5, num_cols=2, sup_title='spatial_clus_dens_1.0E-03', fig_path=fig_path, timeout=timeout, to_remove=['_lm_density_1.00E-01', 'spatial_clus_dens_1.0E-03_'])
    
    
    low_spatial_cluster_exps_and_locs = [
    ('spatial_clus_dens_5.0E-04_p_beacon_0.10_p_no_beacon_0.10_lm_density_1.00E-01', 1),
    ('spatial_clus_dens_5.0E-04_p_beacon_0.10_p_no_beacon_0.50_lm_density_1.00E-01', 2),
    ('spatial_clus_dens_5.0E-04_p_beacon_0.25_p_no_beacon_0.10_lm_density_1.00E-01', 3),
    ('spatial_clus_dens_5.0E-04_p_beacon_0.25_p_no_beacon_0.50_lm_density_1.00E-01', 4),
    ('spatial_clus_dens_5.0E-04_p_beacon_0.50_p_no_beacon_0.10_lm_density_1.00E-01', 5),
    ('spatial_clus_dens_5.0E-04_p_beacon_0.50_p_no_beacon_0.50_lm_density_1.00E-01', 6),
    ('spatial_clus_dens_5.0E-04_p_beacon_0.75_p_no_beacon_0.10_lm_density_1.00E-01', 7),
    ('spatial_clus_dens_5.0E-04_p_beacon_0.75_p_no_beacon_0.50_lm_density_1.00E-01', 8),
    ('spatial_clus_dens_5.0E-04_p_beacon_1.00_p_no_beacon_0.10_lm_density_1.00E-01', 9),
    ('spatial_clus_dens_5.0E-04_p_beacon_1.00_p_no_beacon_0.50_lm_density_1.00E-01', 10),
    ]
    
    plt.figure(figsize=(10,12))
    
    make_summary_plots(low_spatial_cluster_exps_and_locs, num_rows=5, num_cols=2, sup_title='spatial_clus_dens_5.0E-04', fig_path=fig_path, timeout=timeout, to_remove=['_lm_density_1.00E-01', 'spatial_clus_dens_5.0E-04_'])
    
    semantic_2_class_exp_and_loc = [
        ('semantic_num_class_2_p_beacon_0.10_p_no_beacon_0.50_lm_density_1.00E-03', 1),
        ('semantic_num_class_2_p_beacon_0.10_p_no_beacon_0.50_lm_density_1.00E-02', 2),
        ('semantic_num_class_2_p_beacon_0.25_p_no_beacon_0.50_lm_density_1.00E-03', 3),
        ('semantic_num_class_2_p_beacon_0.25_p_no_beacon_0.50_lm_density_1.00E-02', 4),
        ('semantic_num_class_2_p_beacon_0.50_p_no_beacon_0.50_lm_density_1.00E-03', 5),
        ('semantic_num_class_2_p_beacon_0.50_p_no_beacon_0.50_lm_density_1.00E-02', 6),
        ('semantic_num_class_2_p_beacon_0.75_p_no_beacon_0.50_lm_density_1.00E-03', 7),
        ('semantic_num_class_2_p_beacon_0.75_p_no_beacon_0.50_lm_density_1.00E-02', 8),
        ('semantic_num_class_2_p_beacon_1.00_p_no_beacon_0.50_lm_density_1.00E-03', 9),
        ('semantic_num_class_2_p_beacon_1.00_p_no_beacon_0.50_lm_density_1.00E-02', 10),
    ]
    
    plt.figure(figsize=(10,12))
    
    make_summary_plots(semantic_2_class_exp_and_loc, num_rows=5, num_cols=2, sup_title='semantic_num_class_2_p_no_beacon_0.50', fig_path=fig_path, timeout=timeout, to_remove=['_p_no_beacon_0.50', 'semantic_num_class_2_'])
    
    semantic_4_class_exp_and_loc = [
        ('semantic_num_class_4_p_beacon_0.10_p_no_beacon_0.50_lm_density_1.00E-03', 1),
        ('semantic_num_class_4_p_beacon_0.10_p_no_beacon_0.50_lm_density_1.00E-02', 2),
        ('semantic_num_class_4_p_beacon_0.25_p_no_beacon_0.50_lm_density_1.00E-03', 3),
        ('semantic_num_class_4_p_beacon_0.25_p_no_beacon_0.50_lm_density_1.00E-02', 4),
        ('semantic_num_class_4_p_beacon_0.50_p_no_beacon_0.50_lm_density_1.00E-03', 5),
        ('semantic_num_class_4_p_beacon_0.50_p_no_beacon_0.50_lm_density_1.00E-02', 6),
        ('semantic_num_class_4_p_beacon_0.75_p_no_beacon_0.50_lm_density_1.00E-03', 7),
        ('semantic_num_class_4_p_beacon_0.75_p_no_beacon_0.50_lm_density_1.00E-02', 8),
        ('semantic_num_class_4_p_beacon_1.00_p_no_beacon_0.50_lm_density_1.00E-03', 9),
        ('semantic_num_class_4_p_beacon_1.00_p_no_beacon_0.50_lm_density_1.00E-02', 10),
    ]
    
    plt.figure(figsize=(10,12))
    
    make_summary_plots(semantic_4_class_exp_and_loc, num_rows=5, num_cols=2, sup_title='semantic_num_class_4_p_no_beacon_0.50', fig_path=fig_path, timeout=timeout, to_remove=['_p_no_beacon_0.50', 'semantic_num_class_4_'])
    
    semantic_8_class_exp_and_loc = [
        ('semantic_num_class_8_p_beacon_0.10_p_no_beacon_0.50_lm_density_1.00E-03', 1),
        ('semantic_num_class_8_p_beacon_0.10_p_no_beacon_0.50_lm_density_1.00E-02', 2),
        ('semantic_num_class_8_p_beacon_0.25_p_no_beacon_0.50_lm_density_1.00E-03', 3),
        ('semantic_num_class_8_p_beacon_0.25_p_no_beacon_0.50_lm_density_1.00E-02', 4),
        ('semantic_num_class_8_p_beacon_0.50_p_no_beacon_0.50_lm_density_1.00E-03', 5),
        ('semantic_num_class_8_p_beacon_0.50_p_no_beacon_0.50_lm_density_1.00E-02', 6),
        ('semantic_num_class_8_p_beacon_0.75_p_no_beacon_0.50_lm_density_1.00E-03', 7),
        ('semantic_num_class_8_p_beacon_0.75_p_no_beacon_0.50_lm_density_1.00E-02', 8),
        ('semantic_num_class_8_p_beacon_1.00_p_no_beacon_0.50_lm_density_1.00E-03', 9),
        ('semantic_num_class_8_p_beacon_1.00_p_no_beacon_0.50_lm_density_1.00E-02', 10),
    ]
    
    plt.figure(figsize=(10,12))
    
    make_summary_plots(semantic_8_class_exp_and_loc, num_rows=5, num_cols=2, sup_title='semantic_num_class_8_p_no_beacon_0.50', fig_path=fig_path, timeout=timeout, to_remove=['_p_no_beacon_0.50', 'semantic_num_class_8_'])

# for i, exp_df in df.groupby('exp_name'):
#     plt.figure()
#     sns.boxplot(x='planner_type',
#                 y=METRIC_NAME,
#                 hue='planner_size',
#                 data=exp_df)
#     # exp_df.boxplot(column=METRIC_NAME, by=['planner_name'], rot=90)
#     plt.title(exp_df['exp_name'].iloc[0])
    # plt.tight_layout()

In [ ]:

for _, exp_df in df.groupby('exp_name'):
    exp_name = exp_df['exp_name'].iloc[0]
    fig = plt.figure(figsize=(25.6, 9.6))
    plt.subplot(121)
    bins = np.linspace(0, 3, 1000)
    for i, g in exp_df.groupby('planner_id'):
        plt.hist(g[METRIC_NAME], density=True, histtype='step', label=g['planner_name'].iloc[0], bins=bins, cumulative=True)
        
    plt.xlabel('Expected Determinant')
    plt.ylabel('Fraction')
    plt.legend()
    plt.suptitle(exp_name)

    plt.subplot(122)
    exp_configs = configs_by_exp[exp_name]
    plot_mapped_landmarks(exp_configs['mapped_landmarks'], exp_configs['world_map_config'].correlated_beacons.potential)
    plot_road_map(exp_configs['road_map'])
    plt.xlabel('X (m)')
    plt.ylabel('Y (m)')
    plt.tight_layout()
    plt.savefig(fig_path / (exp_name + '.svg'))
    plt.close(fig)
    

In [ ]:
summary_statistics = {}
for keys, sub_df in df.groupby(by=['exp_type', 'planner_type', 'planner_size', 'timeout']):
    # display(sub_df)
    summary_statistics[keys] = {
        r'$p_{25}(E[|\Sigma|])$': sub_df[METRIC_NAME].quantile(0.25),
        r'$p_{50}(E[|\Sigma|])$': sub_df[METRIC_NAME].quantile(0.5),
        r'$p_{75}(E[|\Sigma|])$': sub_df[METRIC_NAME].quantile(0.75),
        r'$p_{50}(T_{plan})$': sub_df['elapsed_time'].quantile(0.50),
        r'$p_{95}(T_{plan})$': sub_df['elapsed_time'].quantile(0.95),
    }

summary_statistics = pd.DataFrame(summary_statistics).transpose()
summary_statistics.index.set_names(['Environment', 'Planner', 'Size', 'Timeout'], inplace=True)



In [ ]:
styler = pd.io.formats.style.Styler(summary_statistics.loc[:, :, :, 3600.0])

In [ ]:
def exp_det_formatter(value):
    if value < 0.01:
        return f"{value:0.2e}"
    else:
        return f"{value:0.3f}"

def index_formatter(value):
    # print(f"'{value}'")
    if value == 'Expected':
        return 'BRULE-E'
    elif value == 'Landmark':
        return "BRULE"
    elif value == 'indep':
        return "Indep."
    elif value == 'mutex':
        return "Mutex"
    elif value == 'semantic':
        return 'Semantic'
    elif value == 'spatial':
        return 'Spatial'
    
    return value

styler.format(subset=[ r'$p_{25}(E[|\Sigma|])$',  r'$p_{50}(E[|\Sigma|])$',  r'$p_{75}(E[|\Sigma|])$'], formatter=exp_det_formatter)
styler.format(subset=[r'$p_{50}(T_{plan})$', r'$p_{95}(T_{plan})$'], precision=3)
styler.format_index(formatter=index_formatter)

In [ ]:
print(styler.to_latex(clines="skip-last;data", hrules=True, column_format='lll|rrr|rr'))

In [ ]:
df

In [ ]:


mask = df['timeout'] == 3600.0
masked_df = df[mask]

out = []

for (planner_type, size), sub_df in masked_df.groupby(['planner_type', 'planner_size']):

    out.append({
        'planner_type': planner_type,
        'size': size,
        'exp_type': 'aggregate',
        'exp_det_p_25': sub_df[METRIC_NAME].quantile(0.25),
        'exp_det_p_50': sub_df[METRIC_NAME].quantile(0.5),
        'exp_det_p_75': sub_df[METRIC_NAME].quantile(0.75),
        'elapsed_time_p_50': sub_df['elapsed_time'].quantile(0.5),
        'elapsed_time_p_95': sub_df['elapsed_time'].quantile(0.95),
    })

    for exp_type, exp_df in sub_df.groupby('exp_type'):
        out.append({
            'planner_type': planner_type,
            'size': size,
            'exp_type': exp_type,
            'exp_det_p_25': exp_df[METRIC_NAME].quantile(0.25),
            'exp_det_p_50': exp_df[METRIC_NAME].quantile(0.5),
            'exp_det_p_75': exp_df[METRIC_NAME].quantile(0.75),
            'elapsed_time_p_50': exp_df['elapsed_time'].quantile(0.5),
            'elapsed_time_p_95': exp_df['elapsed_time'].quantile(0.95),
        })

summary_df = pd.DataFrame(out)

plt.figure(figsize=(8, 4), dpi=100)

color_by_planner_type = {
    'Expected': 'tab:blue',
    'Landmark': 'tab:red',
    'Optimistic': 'tab:green',
}

shape_by_exp_type = {
    'aggregate': '',
    'indep': '*',
    'mutex': '^',
    'spatial': 's',
    'semantic': 'o',
}

line_width_by_exp_type = {
    'aggregate': 2.0,
    'indep': 1,
    'mutex': 1,
    'spatial': 1,
    'semantic': 1,
}

line_style_by_exp_type = {
    'aggregate': 'solid',
    'indep': 'dashed',
    'mutex': 'dashed',
    'spatial': 'dashed',
    'semantic': 'dashed',
}

renamed_df = df.rename(columns={
    'planner_type': 'Planner',
    'exp_type': 'Environment',},
    copy=True,
)
renamed_df.loc[renamed_df['Planner'] == 'Landmark', 'Planner'] = 'BRULE'
renamed_df.loc[renamed_df['Planner'] == 'Expected', 'Planner'] = 'BRULE-E'

plt.subplot(121)
sns.boxenplot(
    data=renamed_df,
    y='planner_size',
    x=METRIC_NAME,
    hue='Planner',
    hue_order=['BRULE', 'BRULE-E', 'Optimistic'],
    legend=False
)
# plt.legend(loc='lower left')
plt.ylabel('Planner Size')
plt.xlabel('Regret')
plt.title('Probability Mass Near Goal')


plt.subplot(122)
sns.boxenplot(
    data=renamed_df,
    y='planner_size',
    x='elapsed_time',
    hue='Planner',
    hue_order=['BRULE', 'BRULE-E', 'Optimistic'],
)

plt.ylabel('')
plt.xlabel('Run Time (s)')
plt.title('Planning Time')
plt.xscale('log')
plt.tight_layout()
# plt.legend(loc='upper right', bbox_to_anchor=(0.165,0.1)
plt.legend(loc='upper right')

plt.savefig('/tmp/sim_results.svg')

plt.figure(figsize=(12, 12), dpi=100)
ax = plt.subplot(221)
df_1 = renamed_df[renamed_df['planner_size']=='1']
sns.boxenplot(data=df_1, x='Environment', y=METRIC_NAME, hue='Planner',
           hue_order=['BRULE', 'BRULE-E', 'Optimistic'],
            formatter=index_formatter,
              
           )
plt.title('1 Sample')
plt.ylabel('Regret')
plt.legend(loc='lower right')


plt.subplot(222, sharey=ax)
df_10 = renamed_df[renamed_df['planner_size']=='10']
sns.boxenplot(data=df_10, x='Environment', y=METRIC_NAME, hue='Planner',
           hue_order=['BRULE', 'BRULE-E'],
            formatter=index_formatter,
           )
plt.title('10 Samples')
plt.ylabel('Regret')
plt.legend(loc='lower right')

plt.subplot(223, sharey=ax)
df_100 = renamed_df[renamed_df['planner_size']=='100']
sns.boxenplot(data=df_100, x='Environment', y=METRIC_NAME, hue='Planner', 
           hue_order=['BRULE', 'BRULE-E'],
            formatter=index_formatter,
           )

plt.title('100 Samples')
plt.ylabel('Regret')
plt.legend(loc='lower right')

plt.subplot(224, sharey=ax)
df_1000 = renamed_df[renamed_df['planner_size']=='1000']
sns.boxenplot(data=df_1000, x='Environment', y=METRIC_NAME, hue='Planner', 
           hue_order=['BRULE', 'BRULE-E'],
            formatter=index_formatter,
           )

plt.title('1000 Samples')
plt.ylabel('Regret')
plt.legend(loc='lower right')
plt.tight_layout()

In [ ]:
brule_mask = df['planner_type'] == "Landmark"
brule_df = df[brule_mask]
brule_df['log_10_prob_mass'] = brule_df['log_prob_mass'] / np.log(10.0)

In [ ]:
plt.figure(figsize=(9,6))
sns.boxenplot(brule_df, x='exp_type', y='log_10_prob_mass', hue='planner_size', showfliers=False)

In [ ]:
for ((planner_size, planner_type), sub_df) in df.groupby(['planner_size', 'planner_type']):
    mean = sub_df['prob_mass_in_region_regret'].mean()
    std = sub_df['prob_mass_in_region_regret'].std()
    num_trials = len(sub_df)
    std_error = std / np.sqrt(num_trials)
    print(f"{planner_type} {planner_size}  {mean: 0.3f} +/-{std_error: 0.3e}")

In [ ]:
fig = plt.figure(figsize=(8, 8))

exps = [
    'indep_p_beacon_0.50_lm_density_1.0E-02_timeout_3600',
    'mutex_num_class_4_lm_density_1.00E-02_timeout_3600',
    'semantic_num_class_2_p_beacon_0.50_p_no_beacon_0.50_lm_density_1.00E-03_timeout_3600',
    'spatial_clus_dens_5.0E-04_p_beacon_0.10_p_no_beacon_0.50_lm_density_1.00E-01_timeout_3600',
    
]

titles = ['Independent', 'Mutex', 'Semantic', 'Spatial']

for i, (exp_name, title) in enumerate(zip(exps, titles)):
    plt.subplot(2,2,i+1)
    exp_configs = configs_by_exp[exp_name]
    plot_road_map(exp_configs['road_map'], label_nodes=False)
    plot_mapped_landmarks(exp_configs['mapped_landmarks'], exp_configs['world_map_config'].correlated_beacons.potential)
    plt.title(title)


fig.supxlabel('X (m)')
fig.supylabel('Y (m)')
plt.tight_layout()
plt.savefig('/tmp/example_environments.svg')

In [ ]:
np.unique(df.exp_name)